# Pipeline complet (Colab)
Ce notebook reproduit les étapes du guide : installation des dépendances, exécution des scripts `train_and_improve.py`, `hyperparameter_optimizer.py`, puis `ensemble_optimizer.py`, et sauvegarde des modèles.

In [ ]:
# 1) Installer les dépendances
!pip install -q pandas numpy scikit-learn xgboost lightgbm optuna tqdm

In [ ]:
# 2) Config Colab : monter Google Drive (optionnel)
USE_DRIVE = False  # passez à True si vous voulez sauvegarder/charger depuis Drive
DRIVE_PROJECT_ZIP = "/content/drive/MyDrive/projet.zip"  # chemin d'un zip du projet si présent dans Drive
TARGET_DIR = "/content/projet"

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

import os
os.makedirs(TARGET_DIR, exist_ok=True)
print(f"TARGET_DIR: {TARGET_DIR}")

In [ ]:
# 3) Importer le projet dans Colab
# Option A: téléverser un zip manuellement (exécuter et choisir un fichier)
# Option B: utiliser un zip présent sur Drive (DRIVE_PROJECT_ZIP)

import shutil
from pathlib import Path

ZIP_PATH = "/content/projet.zip"  # si vous uploadez manuellement

# Décommentez pour uploader manuellement
#from google.colab import files
#uploaded = files.upload()
#if uploaded:
#    ZIP_PATH = f"/content/{list(uploaded.keys())[0]}"

if USE_DRIVE and os.path.exists(DRIVE_PROJECT_ZIP):
    shutil.copy(DRIVE_PROJECT_ZIP, ZIP_PATH)
    print(f"Zip copié depuis Drive: {DRIVE_PROJECT_ZIP}")

if not os.path.exists(ZIP_PATH):
    raise FileNotFoundError("Aucun zip trouvé. Uploadez un zip du projet ou placez-le sur Drive.")

!unzip -o "$ZIP_PATH" -d "$TARGET_DIR" > /dev/null
print("Décompression terminée")

In [ ]:
# 4) Vérifier l'arborescence
import os
for root, dirs, files in os.walk(TARGET_DIR):
    level = root.replace(TARGET_DIR, '').count(os.sep)
    if level > 2:
        continue
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:10]:
        print(f"{subindent}{f}")

In [ ]:
# 5) Chemin vers les scripts
PROJECT_ROOT = TARGET_DIR  # si le zip a créé directement les dossiers
# Ajustez si nécessaire, par ex: PROJECT_ROOT = f"{TARGET_DIR}/Projet"

import pathlib
scripts_dir = pathlib.Path(PROJECT_ROOT) / "scripts"
print(f"Scripts dir: {scripts_dir}")

assert (scripts_dir / "train_and_improve.py").exists(), "train_and_improve.py introuvable"
assert (scripts_dir / "hyperparameter_optimizer.py").exists(), "hyperparameter_optimizer.py introuvable"
assert (scripts_dir / "ensemble_optimizer.py").exists(), "ensemble_optimizer.py introuvable"

In [ ]:
# 6) Exécuter train_and_improve.py (comparaison des modèles)
import subprocess, time, textwrap

# Chemin vers le fichier CSV (ajustez si nécessaire)
CSV_PATH = f"{PROJECT_ROOT}/processed_data/csv_games_fusionnes.csv"

start = time.time()
cmd = ["python", str(scripts_dir / "train_and_improve.py"), "--csv", CSV_PATH]
print("Commande:", " ".join(cmd))
res = subprocess.run(cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
print(f"Durée: {time.time() - start:.1f}s")

In [ ]:
# 7) Exécuter hyperparameter_optimizer.py (Optuna / GridSearch)
RUN_OPTIM = True  # passez à False pour sauter (long)

if RUN_OPTIM:
    start = time.time()
    cmd = ["python", str(scripts_dir / "hyperparameter_optimizer.py"), "--csv", CSV_PATH]
    print("Commande:", " ".join(cmd))
    res = subprocess.run(cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
    print(res.stdout)
    print(res.stderr)
    print(f"Durée: {time.time() - start:.1f}s")
else:
    print("Optimisation sautée (RUN_OPTIM=False)")

In [ ]:
# 8) Exécuter ensemble_optimizer.py (Voting / Stacking)
start = time.time()
cmd = ["python", str(scripts_dir / "ensemble_optimizer.py"), "--csv", CSV_PATH]
print("Commande:", " ".join(cmd))
res = subprocess.run(cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
print(f"Durée: {time.time() - start:.1f}s")

In [ ]:
# 9) Vérifier les sorties (modèles sauvegardés)
import glob
outputs = glob.glob(f"{PROJECT_ROOT}/models/**", recursive=True)
print(f"Total fichiers/sous-dossiers dans models/: {len(outputs)}")
for p in outputs[:40]:
    print(p)


In [ ]:
# 10) Sauvegarde vers Drive (optionnel)
if USE_DRIVE:
    drive_out = "/content/drive/MyDrive/projet_models"
    os.makedirs(drive_out, exist_ok=True)
    !cp -r "$PROJECT_ROOT/models" "$drive_out" 
    print(f"Modèles copiés dans {drive_out}")
else:
    print("Sauvegarde Drive sautée (USE_DRIVE=False)")